In [12]:
from matplotlib import pyplot as plt
from keras.datasets import mnist
import numpy as np
import pandas as pd
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from sklearn.model_selection import train_test_split

In [13]:
train_X = pd.read_csv('train.csv')
test_X = pd.read_csv('test.csv' )
train_X.drop(train_X.columns[train_X.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
test_X.drop(test_X.columns[test_X.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

train_result = pd.read_csv('train_result.csv')
train_y_class = train_result['Class']

X_train = train_X.to_numpy()
X_test = test_X.to_numpy()
train_y = train_y_class.to_numpy()

print(train_y.shape)
print(X_train.shape)
print(X_test.shape)

(50000,)
(50000, 1568)
(10000, 1568)


In [14]:
X_train, X_val, train_y, val_y = train_test_split(X_train, train_y,  test_size=0.20, random_state=101)

In [15]:
print(X_train.shape)
print(X_val.shape)
print(train_y.shape)
print(val_y.shape)

(40000, 1568)
(10000, 1568)
(40000,)
(10000,)


In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=250)

rf_model.fit(X_train, train_y)

RandomForestClassifier(n_estimators=250)

In [17]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [18]:
#

In [19]:
predictions = rf_model.predict(X_val)

rf_matrix = confusion_matrix(val_y,predictions)
print(rf_matrix)

#misclassification_error(confusion_matrix):
sum_preds = np.sum(rf_matrix)
sum_correct = np.sum(np.diag(rf_matrix))
misclassification_error = 1.0 - (float(sum_correct) / float(sum_preds))
    
print("sum_preds               :", int(sum_preds)) 
print("sum_correct_predictions :", int(sum_correct)) 
print("sum_wrong_predictions   :", int(sum_preds - sum_correct))
print("misclassification_error :", misclassification_error)

# measure accuracy
print("Accuracy")
print(accuracy_score(val_y,predictions))

# class-wise accuracy
class_report = classification_report(val_y,predictions)
print(class_report)

[[ 92   0   0   0   0   1   5   0   4   0   0   0   0   0   0   0   0   0
    0]
 [  0 186   3   2   1   1   0   4   0   3   2   0   0   0   0   0   0   0
    0]
 [  3   4 295   4   1   1   8   8  13   2   3   1   0   0   0   0   0   0
    0]
 [  1   7   2 360   3   9   5  12  18  15   1   6   0   0   0   0   0   0
    0]
 [  1   1   4  13 423   3  21   5  16  33  10   4   3   0   0   0   0   0
    0]
 [  1   5   2  12  17 445   7  19  18  19  26  25   1   0   0   0   0   0
    0]
 [  1   1   3   1  18  18 536  12  21  13  22  31   9   2   3   0   1   0
    0]
 [  0   5  10   4   4  26  25 608  24  26  14  24  39   6   0   2   0   0
    0]
 [  0   7   4   9  15   8  26  15 735  16  21  14   6  34   7   0   0   0
    0]
 [  2   1   4   8  13  11  12  30  31 771  27  24  20  12  12   7   2   0
    0]
 [  0   2   3  17  16  23   8  10  38  30 693  23  37   6   7  15   9   1
    0]
 [  0   0   0   1   5  11  28  16  20  57  30 541  11  20   3   7   7   5
    0]
 [  0   0   0   1   1  11  1

In [37]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 20)]
n_estimators = [100, 200, 300]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth = [10, 30, 60, 90]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Criterion
criterion=['gini', 'entropy']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

pprint(random_grid)

{'bootstrap': [True, False],
 'criterion': ['gini', 'entropy'],
 'max_depth': [10, 30, 60, 90, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300]}


In [38]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune

rf_model = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_model, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 30, 60, 90, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, verbose=2)

In [39]:
rf_random.best_params_

{'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 90,
 'criterion': 'gini',
 'bootstrap': False}

In [42]:
rf_best_param_model = RandomForestClassifier(**rf_random.best_params_)

In [43]:
rf_best_param_model.fit(X_train, train_y)

RandomForestClassifier(bootstrap=False, max_depth=90, max_features='sqrt',
                       n_estimators=300)

In [47]:
predictions = rf_best_param_model.predict(X_val)

rf_matrix = confusion_matrix(val_y,predictions)


#misclassification_error(confusion_matrix):
sum_preds = np.sum(rf_matrix)
sum_correct = np.sum(np.diag(rf_matrix))
misclassification_error = 1.0 - (float(sum_correct) / float(sum_preds))
val_accuracy = accuracy_score(val_y,predictions)

In [46]:
print (val_accuracy )

0.7813


# Retrain using whole train dataset and predictions on test set

In [19]:
train_X = pd.read_csv('train.csv')
test_X = pd.read_csv('test.csv' )
train_X.drop(train_X.columns[train_X.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
test_X.drop(test_X.columns[test_X.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

train_result = pd.read_csv('train_result.csv')
train_y_class = train_result['Class']

X_train = train_X.to_numpy()
X_test = test_X.to_numpy()
train_y = train_y_class.to_numpy()

print(train_y.shape)
print(X_train.shape)
print(X_test.shape)

(50000,)
(50000, 1568)
(10000, 1568)


In [20]:
rf_model_final = RandomForestClassifier(n_estimators=300, max_depth=90, min_samples_split=2,
                                     min_samples_leaf=1, bootstrap=False, criterion='gini', max_features='sqrt' )

rf_model_final.fit(X_train, train_y)

RandomForestClassifier(bootstrap=False, max_depth=90, max_features='sqrt',
                       n_estimators=300)

In [21]:
print('\n')
print("Test Predictions")
test_predictions = rf_model.predict(X_test)
print(test_predictions)
print(len(test_predictions))
print(type(test_predictions))
df = pd.DataFrame(test_predictions)
df.index.name= 'Index'
df.columns = ['Class']
file_name = f'test_predictions_random_forest_n300_depth90.csv'
df.to_csv(file_name, index=True)
df



Test Predictions
[10  7 10 ... 10  4  6]
10000
<class 'numpy.ndarray'>


,Class
Index,
0,10
1,7
2,10
3,7
4,5
...,...
9995,12
9996,12
9997,10
